# Imports

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft
from itertools import product

import plotting_results
import adaptive
adaptive.notebook_extension()

import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(20, folder='two_dim_majoranas')
# hpc05.kill_remote_ipcluster()
client, dview, lview = hpc05.start_remote_and_connect(100, folder='two_dim_majoranas')

In [ ]:
import numpy as np
import scipy.constants
import cmath

import functools as ft
import kwant
import sns_system, spectrum, topology, supercurrent


In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Make zigzag shaped system

In [ ]:
params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 1.0341,
                 phase = 0,
                 T = 0.0,
                 V = 0,
                 k_x = 0)

syst_pars = dict(a=10, L_m=500, L_x=2000, W_up=200, W_down=200, z_x=500, z_y=100,
                 offset_lead=100, k_x_in_sc=True, leaded=False, edge_thickness=0)


params = dict(**constants,
              **params_raw)

syst=sns_system.zigzag_system_lead(**syst_pars)
plotting_results.plot_syst(syst=syst, params=dict(**params));

In [ ]:
from itertools import product
discrete_variations = dict()
discrete_variations['z_x'] = [500, 750, 1000, 1250, 1500, 2000, 3000]
discrete_variations['z_y'] = [0, 50, 100, 250, 500, 550, 750, 1000]
discrete_variations['B'] = np.linspace(0, 1, 31)

pars = {}

for z_x, z_y, B in product(*discrete_variations.values()):
    _syst_pars = syst_pars.copy()
    _params = params.copy()
    _syst_pars['z_x'] = z_x
    _syst_pars['L_x'] = z_x*2
    _syst_pars['z_y'] = z_y
    _params['B'] = B
    pars[(z_x, z_y, B)] = (_syst_pars, _params)

In [ ]:
res = lview.map(calc_lowest_state, list(pars.values()))

In [ ]:
res.done()
# res.result()


In [ ]:
data=dict(res.result())

In [ ]:
N_z_x = len(discrete_variations['z_x'])
N_z_y = len(discrete_variations['z_y'])
N_B = len(discrete_variations['B'])
Z_0s = {}
Z_1s = {}

for B in discrete_variations['B']:
    Z_0 = np.zeros((N_z_x, N_z_y))
    Z_1 = np.zeros((N_z_x, N_z_y))
    for idx_x, z_x in enumerate(discrete_variations['z_x']):
        for idx_y, z_y in enumerate(discrete_variations['z_y']):
            key = (z_x, z_y, B)
            Z_0[idx_x, idx_y] = data[key][0]
            Z_1[idx_x, idx_y] = data[key][1]
    Z_0s[B] = Z_0
    Z_1s[B] = Z_1
    

In [ ]:
plot_dict = {('lowest', k): hv.Image(v, bounds=[discrete_variations['z_x'][0], discrete_variations['z_y'][0],
                          discrete_variations['z_x'][-1],discrete_variations['z_y'][-1]]) for k,v in Z_0s.items()}
plot_dict.update({('second lowest', k): hv.Image(v, bounds=[discrete_variations['z_x'][0], discrete_variations['z_y'][0],
                          discrete_variations['z_x'][-1],discrete_variations['z_y'][-1]]) for k,v in Z_0s.items()})

In [ ]:
%%opts Image [colorbar=True] (cmap='Viridis')
# hv.util.Dynamic(hv.HoloMap(plot_dict, kdims=['which eigenmode', 'B']))
(hv.HoloMap(plot_dict, kdims=['which eigenmode', 'B']))

In [ ]:
%opts?

In [ ]:
print(res.done(), res.started, res.elapsed)


In [ ]:
def bands_wrap(k):
    ham = syst.hamiltonian_submatrix(sparse=True, params=dict(**params, k_x=k))
    e, ev = spectrum.sparse_diag(ham, 100, 0)
    return np.sort(e)
    
learner = adaptive.Learner1D(bands_wrap, [-np.pi, np.pi])
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
ham = syst.hamiltonian_submatrix(params=dict(**params, k_x=0), sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.sparse_diag(ham,4,0)

In [ ]:
e

In [ ]:
ax=plt.axes()
plotting_results.plot_syst(syst=syst, params=dict(**params, k_x=0),ax=ax)
kwant.plotter.map(syst, d(ev[:,0]), ax=ax)
# plt.savefig('no_ohase_difference', dpi=1000)